<a href="https://colab.research.google.com/github/luciedum-dot/Intro-to-data-analysis-with-Python---Final-Project/blob/main/Evolutions_of_Criminality_France_2016_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>